# Data wrangling and datetimes

Dates and times are a special kind of data type. In this notebook, we will keep exploring the `orders` and `orderlines` datasets from Eniac and learn to deal with them.

In [1]:
import pandas as pd
df = pd.read_csv('data/orderlines.csv', parse_dates=['date'])
df.head(15)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11


## Data exploration

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


Two variables need to be modified: 

* `unit_price`: it is detected as an object but it has to be a float. Why is that happening? 
* `date`: needs to be transformed tp a date time format. 

For the moment we will only focus on the `date`. Since it contains both the date and the time, we will transform the data type using the pandas method `to_datetime`: 

In [3]:
df.unit_price  = pd.to_numeric(df.unit_price,errors='coerce' )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        257814 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


You can count the quantity of observations with `.value_counts()`.

In [4]:
df['sku'].value_counts()

MIC0036      6282
APP1216      5627
APP0662      5445
APP1190      5039
APP0663      3942
             ... 
APP2110         1
TWS0053-A       1
APP1837         1
UAG0026         1
REP0299         1
Name: sku, Length: 7951, dtype: int64

Exercise: check if the dates from the `orderlines` dataset match with the dates on the `orders` dataset.

In [5]:
# code here
df_orders = pd.read_csv('data/orders.csv', parse_dates=['created_date'], index_col='created_date')
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 226909 entries, 2017-01-02 13:35:40 to 2018-03-14 13:58:36
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   order_id    226909 non-null  int64  
 1   total_paid  226904 non-null  float64
 2   state       226909 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 6.9+ MB


In [6]:
df.iloc[df.index.isin(df_orders.index)==True]

,id,id_order,product_id,product_quantity,sku,unit_price,date


## Aggregating with pandas

* Grouping and aggregating is one of the main ways to explore data. The main tools to do that with pandas are:
    * [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).
    * [`pandas.DataFrame.agg()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html?highlight=agg#pandas.DataFrame.agg) 
        * [`pandas.DataFrame.count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)

In [7]:
df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


How many id do we have on each sku?

In [8]:
df.groupby(['sku']).agg({'product_quantity':'count'})

,product_quantity
sku,
8MO0001-A,2
8MO0003-A,3
8MO0007,29
8MO0008,30
8MO0009,28
...,...
ZAG0041,2
ZAG0042,1
ZEP0007,5


Which is the total quantity acquired for each `sku`? And the median? And the mean?

In [9]:
df.groupby(['sku']).agg({'product_quantity':['count', 'sum','median','mean']})

product_quantity                     
                     count sum median      mean
sku                                            
8MO0001-A                2   2    1.0  1.000000
8MO0003-A                3   3    1.0  1.000000
8MO0007                 29  30    1.0  1.034483
8MO0008                 30  31    1.0  1.033333
8MO0009                 28  30    1.0  1.071429
...                    ...  ..    ...       ...
ZAG0041                  2   2    1.0  1.000000
ZAG0042                  1   1    1.0  1.000000
ZEP0007                  5   5    1.0  1.000000
ZEP0008                  1   1    1.0  1.000000
par0072                 12  13    1.0  1.083333

[7951 rows x 4 columns]

`groupby` can also be combined with other pandas functions to analyse in more depth the datasets.

In [10]:
df.groupby('sku')[['sku','product_quantity']].describe()

product_quantity                                             
                     count      mean       std  min  25%  50%  75%  max
sku                                                                    
8MO0001-A              2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0003-A              3.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0007               29.0  1.034483  0.185695  1.0  1.0  1.0  1.0  2.0
8MO0008               30.0  1.033333  0.182574  1.0  1.0  1.0  1.0  2.0
8MO0009               28.0  1.071429  0.262265  1.0  1.0  1.0  1.0  2.0
...                    ...       ...       ...  ...  ...  ...  ...  ...
ZAG0041                2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZAG0042                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
ZEP0007                5.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZEP0008                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
par0072               12.0  1.083333  0.288675  1.0  1.0  1.0  1.0  2.0

[7951 rows x 8 columns]

Now let's see how can I combine `.groupby()` and `.aggragate()` with `.sort_values()`.

I would like to see the top 10 most sold products on our data (total product quantity). 

In [11]:
(df
    .groupby(['sku'])
    .agg({'product_quantity': 'sum','unit_price':'sum'})
    .sort_values(by ='product_quantity', ascending=False).head(10)
)

,product_quantity,unit_price
sku,,
APP1190,6366,283187.77
MIC0036,6316,421087.00
APP1216,5648,733997.57
APP0662,5487,803976.85
APP0663,4164,812144.91
MMW0016,2615,54111.01
APP0698,2348,16411.22
SAT0054,2322,161853.15
APP1214,1985,150338.42


How can I sort values with multiple aggregated functions?

In [12]:
(
df
    .groupby('sku')
    .agg({'product_quantity':['sum','count','std','mean','median']})
    .sort_values(('product_quantity','mean'), ascending=False)
)

product_quantity                                    
                       sum count         std       mean median
sku                                                           
APP1917                 32     1         NaN  32.000000   32.0
ADN0039                132     7   47.245559  18.857143    1.0
KIN0137                862    55  107.718263  15.672727    1.0
EVU0013                177    12   47.005077  14.750000    1.0
SEV0028               1122   122   90.353268   9.196721    1.0
...                    ...   ...         ...        ...    ...
APP1546                  1     1         NaN   1.000000    1.0
APP1546-A                4     4    0.000000   1.000000    1.0
MAK0009                  2     2    0.000000   1.000000    1.0
MAK0008                  1     1         NaN   1.000000    1.0
8MO0001-A                2     2    0.000000   1.000000    1.0

[7951 rows x 5 columns]

## Working with date time

Time to create the week day column. You will have to combine two functions, [`pandas.DataFrame.assign()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) from `pandas` and [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) from `datetime`.

First, let's talk about `.assign()`. It is a useful method for creating new columns:

In [13]:
df.assign(new_colum = 'hi! I am a new column!').head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,new_colum
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,hi! I am a new column!
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,hi! I am a new column!
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,hi! I am a new column!
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,hi! I am a new column!
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,hi! I am a new column!


A new column often is the result of an operation between other columns in the dataframe:

In [14]:
df.assign(total_price = df['product_quantity'] * df['unit_price']).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74


In Pandas, we can extract strings containing elements like the day of the month, the day of the week, the hour of the day... from `datetime` objects. We do so by using `dt.strftime()` in combination with `strftime` syntax. Find the cheat sheet for this syntax [here](https://strftime.org/). 

You'll understand it better with an example. Let's create the weekday column:

In [15]:
df = df.assign(week_day = df['date'].dt.day_name())
df

,id,id_order,product_id,product_quantity,sku,unit_price,date,week_day
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,Sunday
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,Sunday
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,Sunday
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,Sunday
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,Sunday
...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,Wednesday
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,Wednesday
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,Wednesday
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,Wednesday


Now we can find out which are the week day with more sells in quantity: 

In [16]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby('week_day')
    .agg({'product_quantity':['sum','count']})
    .sort_values(('product_quantity','sum'), ascending=False)
)

product_quantity       
                       sum  count
week_day                         
Monday               57624  50307
Wednesday            54131  47550
Tuesday              50871  44498
Friday               49566  44027
Thursday             48431  43634
Sunday               35667  32857
Saturday             33302  31110

Is there a way to find out the top 1 product (in product quantity) by each sku in each weekday?

In [17]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby(['week_day','sku'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('week_day')
    .head(1)
)

,week_day,sku,product_quantity
0,Friday,APP1190,1827
1,Thursday,MIC0036,1389
3,Tuesday,MIC0036,1213
6,Wednesday,SEV0028,1014
10,Monday,APP0663,840
14,Saturday,MIC0036,806
23,Sunday,APP1190,692


# CHALLENGES

1. Which are the top 6 orders with a higher number of products sold?

In [18]:
(
df
    .groupby('id_order')
    .agg({'product_quantity':['max']})
    .sort_values(('product_quantity','max'), ascending=False)
).head(6)

,product_quantity
,max
id_order,
358747,999
346221,999
349475,800
349133,555
335057,201
417536,200


2. Which are the top 6 orders for the 1 of July of 2017? 

Combining `.assign()` with `.dt.date` will help you extract the date of a datetime column and use this information to filter it. Do some googleing to find out how to use `.dt.date`.

Here is an example how to filter the first of January 2017.

In [19]:
df_date = df.assign(date = df['date'].dt.date).copy()
top_6 = df_date[df_date['date'] == pd.to_datetime('2017-07-01')]

/opt/miniconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [20]:
df1 = df.copy()
df1.set_index('date',inplace=True)
df1

,id,id_order,product_id,product_quantity,sku,unit_price,week_day
date,,,,,,,
2017-01-01 00:07:19,1119109,299539,0,1,OTT0133,18.99,Sunday
2017-01-01 00:19:45,1119110,299540,0,1,LGE0043,399.00,Sunday
2017-01-01 00:20:57,1119111,299541,0,1,PAR0071,474.05,Sunday
2017-01-01 00:51:40,1119112,299542,0,1,WDT0315,68.39,Sunday
2017-01-01 01:06:38,1119113,299543,0,1,JBL0104,23.74,Sunday
...,...,...,...,...,...,...,...
2018-03-14 13:57:25,1650199,527398,0,1,JBL0122,42.99,Wednesday
2018-03-14 13:57:34,1650200,527399,0,1,PAC0653,141.58,Wednesday
2018-03-14 13:57:41,1650201,527400,0,2,APP0698,9.99,Wednesday


In [21]:

(
df1.loc['2017-07-01']
    .groupby('id_order')
    .agg({'product_quantity':['sum']})
    .sort_values(('product_quantity','sum'), ascending=False)
).head(6)
                     

,product_quantity
,sum
id_order,
371355,8
371217,7
371309,7
371285,5
371178,5
371120,5


3. Which is the month with the highest number of units sold? Remember to look at the documentation of [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) 

## option 1

In [22]:

df_month = df.assign(date = df['date'].dt.strftime('%B'))


(
df_month
    .groupby('date')
    .agg({'product_quantity':['max']})
    .sort_values(('product_quantity','max'), ascending=False)
).head(6)


,product_quantity
,max
date,
April,999
May,999
March,201
October,200
December,192
January,164


## option 2

In [23]:
df1.resample('M')['product_quantity'].max().sort_values(ascending=False)

date
2017-04-30    999
2017-05-31    999
2017-03-31    201
2017-10-31    200
2017-12-31    192
2018-01-31    164
2017-02-28    126
2018-03-31    125
2017-06-30     99
2017-01-31     92
2017-08-31     72
2017-11-30     72
2017-09-30     41
2018-02-28     37
2017-07-31     27
Name: product_quantity, dtype: int64

The results enormous differences between some months. Why do you think this is happening? Do we only have one year of data? In case you have multiple years, filter for only 2017 to find out the best month for that year.

ATTENTION: the output of the function `.dt.strftime()` is a string!

## option 1

In [24]:


df_2017 = df.assign(date_year = df['date'].dt.strftime('%Y'))
df_2017_month = df_2017.assign(date_month = df_2017['date'].dt.strftime('%B'))

df_by_month = df_2017_month[df_2017_month['date_year'] == '2017']

In [25]:

(
df_by_month
    .groupby('date_month')
    .agg({'product_quantity':['count']})
    .sort_values(('product_quantity','count'), ascending=False)
).head(6)


,product_quantity
,count
date_month,
November,42924
December,36393
January,21557
July,16030
October,15736
February,14149


## option 2

In [26]:


df1 = df1.assign(total_price = df1['product_quantity'] * df1['unit_price'])



In [27]:
df1.loc['2017'].resample('M')['total_price'].max().sort_values(ascending=False)

date
2017-12-31    99648.00
2017-06-30    87021.00
2017-04-30    55934.01
2017-08-31    27545.76
2017-05-31    22881.29
2017-11-30    22672.00
2017-03-31    18326.00
2017-02-28    15959.43
2017-09-30    15030.19
2017-10-31    14706.60
2017-07-31    11204.73
2017-01-31     4379.94
Name: total_price, dtype: float64

4. Find the day of the week with the highest amount of products sold by each MONTH. See an idea of the expected output. Take into account the year problem we mentioned before.

In [28]:
 pd.DataFrame({
    'month':['November','January','July','...'],
    'week_day':['Friday','Tuesday','Monday','...'],
    'product_quantity':[10000, 9800, 9700, 0]
})


,month,week_day,product_quantity
0,November,Friday,10000
1,January,Tuesday,9800
2,July,Monday,9700
3,...,...,0


In [29]:
df_by_month=df1.loc['2017'].resample('D').agg({'product_quantity':'count'}).sort_values('product_quantity',ascending=False)
#['product_quantity'].sum().sort_values(ascending=False)

In [30]:
df_by_month.iloc[::]

,product_quantity
date,
2017-11-24,7040
2017-11-23,4514
2017-11-27,4382
2017-11-25,3357
2017-11-26,3274
...,...
2017-04-15,200
2017-06-17,199
2017-06-24,199


In [31]:
df_year =  df.assign(date_year = df['date'].dt.strftime('%Y'))
df_weekday = df_year.assign(date_week_day = df['date'].dt.strftime('%A'))

In [32]:
df_weekday_month = df_weekday.assign(date_month = df['date'].dt.strftime('%B'))
df_weekday_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        257814 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   week_day          293983 non-null  object        
 8   date_year         293983 non-null  object        
 9   date_week_day     293983 non-null  object        
 10  date_month        293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


In [33]:

(
df_weekday_month[df_weekday_month['date_year'] == '2017']
    .groupby(['date_month','date_week_day'])
    .agg({'product_quantity':['count']})
    .sort_values(('product_quantity','count'), ascending=False)
    
).reset_index()

,date_month,date_week_day,product_quantity
,,,count
0,November,Friday,8852
1,November,Thursday,7025
2,November,Wednesday,6755
3,December,Wednesday,6564
4,November,Monday,6400
...,...,...,...
79,March,Saturday,1038
80,May,Sunday,1019
81,August,Saturday,983
